# Visualize waveforms generated for the AWG

This notebook is used to quickly visualize waveforms that are uploaded to the AWG. Figures generated here can also be saved and used in publication supplementary.

Dinesh Pinto (d.pinto@fkf.mpg.de)

In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# Enter name of pkl file without channel names
waveform_name = 'pulsedODMR' 
folder = os.path.join("C:\\", "qudi-hira", "saved_pulsed_assets", "waveform")

In [3]:
os.listdir(folder)

['cwODMR_a_ch0.pkl', 'cwODMR_a_ch1.pkl', 'cwODMR_d_ch0.pkl', 'cwODMR_d_ch1.pkl', 'cwODMR_d_ch2.pkl', 'cwODMR_d_ch3.pkl', 'HILO50_a_ch0.pkl', 'HILO50_a_ch1.pkl', 'HILO50_a_ch2.pkl', 'HILO50_a_ch3.pkl', 'HILO50_d_ch0.pkl', 'HILO50_d_ch1.pkl', 'HILO50_d_ch2.pkl', 'HILO50_d_ch3.pkl', 'HILO50_d_ch4.pkl', 'HILO50_d_ch5.pkl', 'HILO_a_ch0.pkl', 'HILO_a_ch1.pkl', 'HILO_a_ch2.pkl', 'HILO_a_ch3.pkl', 'HILO_d_ch0.pkl', 'HILO_d_ch1.pkl', 'HILO_d_ch2.pkl', 'HILO_d_ch3.pkl', 'HILO_d_ch4.pkl', 'HILO_d_ch5.pkl', 'laser_empty_a_ch0.pkl', 'laser_empty_a_ch1.pkl', 'laser_empty_a_ch2.pkl', 'laser_empty_a_ch3.pkl', 'laser_empty_d_ch0.pkl', 'laser_empty_d_ch1.pkl', 'laser_empty_d_ch2.pkl', 'laser_empty_d_ch3.pkl', 'laser_empty_d_ch4.pkl', 'laser_empty_d_ch5.pkl', 'ODMR_a_ch0.pkl', 'ODMR_a_ch1.pkl', 'ODMR_d_ch0.pkl', 'ODMR_d_ch2.pkl', 'ODMR_d_ch5.pkl', 'pulsedODMR_a_ch0.pkl', 'pulsedODMR_a_ch1.pkl', 'pulsedODMR_d_ch0.pkl', 'pulsedODMR_d_ch1.pkl', 'pulsedODMR_d_ch2.pkl', 'pulsedODMR_iq2_a_ch0.pkl', 'pulsedODMR

In [5]:
#%matplotlib notebook

keys = ['a_ch0', 'a_ch1', 'a_ch2', 'a_ch3', 'd_ch0', 'd_ch1', 'd_ch2', 'd_ch3', 'd_ch4', 'd_ch5']
labels = {
    'a_ch0': "In-phase", 
    'a_ch1': "Quadrature", 
    'a_ch2': "", 
    'a_ch3': "", 
    'd_ch0': "Laser", 
    'd_ch1': "Readout", 
    'd_ch2': "Switch", 
    'd_ch3': "", 
    'd_ch4': "", 
    'd_ch5': ""
}

# Initialize all channels to None
channels = dict.fromkeys(keys, None) 

for key in list(channels):
    try:
        filename = f"{waveform_name}_{key}.pkl"
        filepath = os.path.join(folder, filename)
        with open(filepath, 'rb') as f:
            channels[key] = pickle.load(f)
        if not np.any(channels[key]):
            # Delete channels with only zeros
            print(f'"{key}" is a null channel.')
            del channels[key]  
    except FileNotFoundError as exc:
        # Delete channels that don't exist on disk
        print(exc) 
        del channels[key]


if len(channels) == 1: # Single channel plotting
    size_x, size_y = 8, 5
else:
    size_x, size_y = len(channels) * 1.5, len(channels) * 1.5

fig, ax = plt.subplots(nrows=len(channels), sharex=True, figsize=(size_x, size_y))

# Correct for matplotlib's (silly) behavior for single plots
if not isinstance(ax, np.ndarray): 
    ax = [ax]

    
limit = 0

for idx, key in enumerate(channels):
    #time_in_us = np.linspace(0, len(channels[key][:limit]) / 1.25e9, len(channels[key][limit:])) * 1e6
    time_in_us = np.linspace(0, len(channels[key][limit:]) / 1.25e9, len(channels[key][limit:])) * 1e6
    
    if key.startswith('a_'):
        ax[idx].plot(time_in_us, channels[key][limit:], ".-", linewidth=0.2, color="C0")
    elif key.startswith('d_ch1'):
        ax[idx].plot(time_in_us, channels[key][limit:], "-", linewidth=0.2, color="C0")
    else:
        ax[idx].plot(time_in_us, channels[key][limit:],"-", linewidth=0.5, color="C1")
    ax[idx].set_title(f'{key} ({labels[key]})')

fig.tight_layout()
fig.savefig(os.path.join(folder, "pODMR.png"), dpi=600)

[Errno 2] No such file or directory: 'C:\\qudi-hira\\saved_pulsed_assets\\waveform\\pulsedODMR_a_ch2.pkl'
[Errno 2] No such file or directory: 'C:\\qudi-hira\\saved_pulsed_assets\\waveform\\pulsedODMR_a_ch3.pkl'
[Errno 2] No such file or directory: 'C:\\qudi-hira\\saved_pulsed_assets\\waveform\\pulsedODMR_d_ch3.pkl'
[Errno 2] No such file or directory: 'C:\\qudi-hira\\saved_pulsed_assets\\waveform\\pulsedODMR_d_ch4.pkl'
[Errno 2] No such file or directory: 'C:\\qudi-hira\\saved_pulsed_assets\\waveform\\pulsedODMR_d_ch5.pkl'


Traceback (most recent call last):
  File "C:\Users\qudiamond\AppData\Local\Continuum\anaconda3\envs\qudi\lib\site-packages\matplotlib\backends\backend_agg.py", line 171, in draw_path
    self._renderer.draw_path(gc, path, transform, rgbFace)
OverflowError: In draw_path: Exceeded cell block limit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\qudi-hira\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-abb706faed14>", line 63, in <module>
    fig.savefig(os.path.join(folder, "pODMR.png"), dpi=600)
  File "C:\Users\qudiamond\AppData\Local\Continuum\anaconda3\envs\qudi\lib\site-packages\matplotlib\figure.py", line 2035, in savefig
    self.canvas.print_figure(fname, **kwargs)
  File "C:\Users\qudiamond\AppData\Local\Continuum\anaconda3\envs\qudi\lib\site-packages\matplotlib\backend_bases.py", line 2263, in print_figure
    **kwargs)
  

In [ ]:
%matplotlib notebook

keys = ['a_ch0', 'a_ch1', 'a_ch2', 'a_ch3', 'd_ch0', 'd_ch1', 'd_ch2', 'd_ch3', 'd_ch4', 'd_ch5']
labels = {
    'a_ch0': "In-phase", 
    'a_ch1': "Quadrature", 
    'a_ch2': "", 
    'a_ch3': "", 
    'd_ch0': "Laser", 
    'd_ch1': "Readout", 
    'd_ch2': "Switch", 
    'd_ch3': "", 
    'd_ch4': "", 
    'd_ch5': ""
}

# Initialize all channels to None
channels = dict.fromkeys(keys, None) 

for key in list(channels):
    try:
        filename = f"{waveform_name}_{key}.pkl"
        filepath = os.path.join(folder, filename)
        with open(filepath, 'rb') as f:
            channels[key] = pickle.load(f)
        if not np.any(channels[key]):
            # Delete channels with only zeros
            print(f'"{key}" is a null channel.')
            del channels[key]  
    except FileNotFoundError as exc:
        # Delete channels that don't exist on disk
        print(exc) 
        del channels[key]


if len(channels) == 1: # Single channel plotting
    size_x, size_y = 8, 5
else:
    size_x, size_y = len(channels) * 1.5, len(channels) * 1.5

fig, ax = plt.subplots(nrows=len(channels), sharex=True, figsize=(size_x, size_y))

# Correct for matplotlib's (silly) behavior for single plots
if not isinstance(ax, np.ndarray): 
    ax = [ax]

    
limit = 0

for idx, key in enumerate(channels):
    #time_in_us = np.linspace(0, len(channels[key][:limit]) / 1.25e9, len(channels[key][limit:])) * 1e6
    time_in_us = np.linspace(0, len(channels[key][limit:]), len(channels[key][limit:]))
    
    if key.startswith('a_'):
        ax[idx].plot(time_in_us, channels[key][limit:], ".-", linewidth=0.2, color="C0")
    elif key.startswith('d_ch1'):
        ax[idx].plot(time_in_us, channels[key][limit:], "-", linewidth=0.2, color="C0")
    else:
        ax[idx].plot(time_in_us, channels[key][limit:],"-", linewidth=0.5, color="C1")
    ax[idx].set_title(f'{key} ({labels[key]})')

fig.tight_layout()